# Pain Charts classification with CNN

Idea: Creation of a simple a classifier for the groups found in Boudreau et al 2018

- Data from Patellofemoral Pain Patients
- Groups defined by a unsupervised cluster classfication (K-means)

Method: Application of a CNN with a simple architecture and build up from here
     
Reference: Boudreau, S.A., Royo, A.C., Matthews, M. et al. Distinct patterns of variation in the distribution of knee pain. Sci Rep 8, 16522 (2018). www.nature.com/articles/s41598-018-34950-2</div><i class="fa fa-lightbulb-o "></i>

In [3]:
#Import of the the main 
import sklearn
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import glob
import cv2


import tensorflow as tf

from keras import layers
from keras.layers import Input, Add, Dense, Activation,ZeroPadding2D, ZeroPadding1D, BatchNormalization, Flatten, Conv1D, Conv2D, AveragePooling2D, AveragePooling1D,MaxPooling2D, MaxPooling1D, GlobalMaxPooling1D,GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils, to_categorical   
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import lmdb
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [4]:
# Let's import the file names from our dataset
fileNames = glob.glob(r'C:\Users\Albert\Documents\GitHub\PC data\*.png')
print('Number of files imported is',len(fileNames))
kernel = np.array([[0,1,0],[1,1,1],[0,1,0]],np.uint8)
binIms = []   
PainChartList = []

RawDataList = []

for fileName in fileNames:
    im = cv2.imread(fileName)
    RawDataList.append(im)
    binIm = np.zeros(np.shape(im)[0:2])
    #Creating a mask from the pain Image, this will be the binarized version of the image
    binIm[(im[:,:,2]>150) & (im[:,:,0]==0)] = 1
    #Open the pain region so small empty spaces within the pain region are filled
    binIm = cv2.morphologyEx(binIm, cv2.MORPH_CLOSE, kernel)
    #Saving images in a List 
    PainChartList.append(binIm)

Number of files imported is 299


In [5]:
y = np.loadtxt('C:/Users/Albert/Documents/GitHub/classesK4')

In [6]:
arrayPain = np.asarray(PainChartList)
arrayPainCropped = arrayPain[:,int(arrayPain.shape[1]/2):int(arrayPain.shape[1]),:]
X_train, X_test, Y_train, Y_test = train_test_split(arrayPainCropped, y, test_size=0.3)
X_trainR = np.expand_dims(X_train,3)
X_testR = np.expand_dims(X_test,3)

In [7]:
def PainNet(input_shape = (568, 447, 1), classes = 4):
    """
    Implementation of PainNet the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL 
    
    Arguments:
    input_shape -- shape of the images of the dataset (?,568, 447, 1)
    classes -- integer, number of classes (?,4)

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
   
    # Stage 1
    X = Conv2D(32, (30,30), strides = (3, 3), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X) #(228, 168, 64)
    print('Conv1:',X.shape)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X) #114, 84, 64
    #print('Pool1:',X.shape)
    
    # Stage 2
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv2', kernel_initializer = glorot_uniform(seed=0))(X) #(50,35,128)
    print('Conv2:',X.shape)
    X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    #print('Pool2:',X.shape)
    
     # Stage 2
    X = Conv2D(128, (2, 2), strides = (2, 2), name = 'conv3', kernel_initializer = glorot_uniform(seed=0))(X) #(50,35,128)
    print('Conv2:',X.shape)
    X = BatchNormalization(axis = 3, name = 'bn_conv3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    #print('Pool2:',X.shape)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='PainNet')

    return model

In [8]:
model = PainNet(input_shape = X_trainR.shape[1:], classes = 4)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Conv1: (None, 182, 142, 32)
Conv2: (None, 42, 32, 64)
Conv2: (None, 10, 7, 128)


In [9]:
Y_train = to_categorical(Y_train.T, num_classes=4)
Y_test = to_categorical(Y_test.T, num_classes=4)
print ("number of training examples = " + str(X_trainR.shape[0]))
print ("number of test examples = " + str(X_testR.shape[0]))
print ("X_trainR shape: " + str(X_trainR.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_testR shape: " + str(X_testR.shape))
print ("Y_test shape: " + str(Y_test.shape))
conv_layers = {}

number of training examples = 209
number of test examples = 90
X_trainR shape: (209, 568, 447, 1)
Y_train shape: (209, 4)
X_testR shape: (90, 568, 447, 1)
Y_test shape: (90, 4)


In [10]:
#model.fit(X_trainR,Y_train,epochs = 5, batch_size = 16, shuffle = True)

# Tranfer learning from ResNet50 

In [89]:
from keras.applications import ResNet50,MobileNet 
from keras import backend as K

In [97]:
#Loading ResNet50 excluding the last layers
base_model = ResNet50 (weights = 'imagenet',include_top = False,input_shape=(568, 447, 3))

C:\Users\Albert\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [98]:
x=base_model.output #Loading the output of the base model and including new leayers
x= Flatten()(x)
#x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(4,activation='softmax')(x) #final layer with softmax activation

In [99]:
#Architecture of the new Model accomplished
new_model=Model(inputs=base_model.input,outputs=preds)
#set the first 30 layers of the network to be non-trainable
for layer in model.layers[1:30]:
    layer.trainable=False
for layer in model.layers[30:]:
    layer.trainable=True

In [55]:
arrayPain2 = np.asarray(RawDataList)
arrayPainCropped2 = arrayPain2[:,int(arrayPain2.shape[1]/2):int(arrayPain2.shape[1]),:,:]
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(arrayPainCropped2, y, test_size=0.3)
Y_train2 = to_categorical(Y_train2.T, num_classes=4)
Y_test2 = to_categorical(Y_test2.T, num_classes=4)
print ("number of training examples = " + str(X_train2.shape[0]))
print ("number of test examples = " + str(X_test2.shape[0]))
print ("X_train2 shape: " + str(X_train2.shape))
print ("Y_train2 shape: " + str(Y_train2.shape))
print ("X_test2 shape: " + str(X_test2.shape))
print ("Y_test2 shape: " + str(Y_test2.shape))

number of training examples = 209
number of test examples = 90
X_train2 shape: (209, 568, 447, 3)
Y_train2 shape: (209, 4)
X_test2 shape: (90, 568, 447, 3)
Y_test2 shape: (90, 4)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train2)


#compile the model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# fits the model on batches with real-time data augmentation:
new_model.fit_generator(datagen.flow(X_train2,Y_train2, batch_size=32),
                    steps_per_epoch=len(X_train2) / 32, epochs=10)


Epoch 1/10


In [71]:
new_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________